<h1> Examen Deep Learning </h1>

Examen de 3 heures, sur machine, tout documents autorisÃ©s. Pour l'anonymisation de votre rendu, veuillez suivre les consignes : 

- Choisissez un numÃ©ro alÃ©atoire Ã  6 chiffres, le plus alÃ©atoire possible..
- Ecrivez ce numÃ©ro sur une feuille, ainsi que votre nom et numÃ©ro d'Ã©tudiant 
- Pliez cette feuille en 4, et donnez la Ã  la fin de l'examen au moment d'Ã©marger  
- Zippez votre fichier notebook, et nommez l'archive avec votre numÃ©ro alÃ©atoire 
- Envoyez l'archive via la page : http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/upload/upload.php

<hr/>

Un musÃ©e spÃ©cialisÃ© en peintures reprÃ©sentant des animaux de la savane a malheureusement perdu son fichier d'inventaire qui regroupait des informations prÃ©cieuses sur les 2000 oeuvres du musÃ©e ! Fort heureusement, l'informaticien toujours prÃ©voyant avait conservÃ© une copie des 2000 photos des oeuvres, et il vient juste de suivre une formation de Deep Learning ...

<center>
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/0.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/1.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/2.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/3.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/4.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/5.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/6.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/7.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/8.jpg" width="90px" />
<img src=" http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/9.jpg" width="90px" />
</center>

Cet examen porte sur la rÃ©solution d'une tÃ¢che de classification d'images sur un jeu de donnÃ©es faiblement annotÃ©. Plusieurs solutions sont envisagÃ©es  consistant Ã  tirer parti d'annotations de classes Ã©quivalentes sur un autre dataset entiÃ¨rement Ã©tiquettÃ© (imagenet). Les deux jeux de donnÃ©es contiennent 2000 images rÃ©parties (Ã©galement) en quatre classes (d'animaux : zÃ¨bres, gorilles, lÃ©opards, tigres). 1500 images sont utilisÃ©es pour l'entrainement, 500 pour l'Ã©valuation. Seul le jeu de donnÃ©es A (imagenet) est complÃ¨tement annotÃ©, le dataset B (les photos des oeuvres du musÃ©e) ne contient que trÃ¨s peu d'annotations.

La partie 1 consiste Ã  entrainer et Ã©valuer les performances d'un modÃ¨le CNN fourni sur les deux datasets d'images. Les parties 2 et 3 consistent Ã  mettre en oeuvre deux solutions par <i>adaptation de domaines</i> pour tenter d'amÃ©liorer les performances de classification sur le dataset faiblement annotÃ©. Vous pouvez consacrer environ une heure Ã  chaque partie, notÃ©es Ã©galement.


<h3> TÃ©lÃ©chargement des donnÃ©es </h3>

In [0]:
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/imagenetXtrain.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/imagenetYtrain.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/imagenetXtest.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/imagenetYtest.npy
  
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/awaXtrain.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/awaYtrain.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/awaXtest.npy
!wget http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/awaYtest.npy  

<h3>Chargement des donnÃ©es</h3>

In [0]:
import numpy as np
from random import shuffle
import keras
import keras.backend as K
from keras.layers import Input, Dense, Flatten, LeakyReLU
from keras.layers import AveragePooling2D, Conv2D, Activation, BatchNormalization, Dropout
from keras.models import Model
from keras.optimizers import Adam

In [0]:
path = './'
num_classes = 4

x_trainA = np.load(path+'imagenetXtrain.npy')
x_trainA = x_trainA / 255.
y_trainA = np.load(path+'imagenetYtrain.npy')
y_trainA = keras.utils.to_categorical(y_trainA, num_classes)

x_testA = np.load(path+'imagenetXtest.npy')
x_testA = x_testA / 255.
y_testA = np.load(path+'imagenetYtest.npy')
y_testA = keras.utils.to_categorical(y_testA, num_classes)

x_trainB = np.load(path+'awaXtrain.npy')
x_trainB = x_trainB / 255.
y_trainB = np.load(path+'awaYtrain.npy')
y_trainB = keras.utils.to_categorical(y_trainB, num_classes)

x_testB = np.load(path+'awaXtest.npy')
x_testB = x_testB / 255.
y_testB = np.load(path+'awaYtest.npy')
y_testB = keras.utils.to_categorical(y_testB, num_classes)

print('x_trainA.shape:', x_trainA.shape, 'y_trainA.shape:', y_trainA.shape)
print('x_trainB.shape:', x_trainB.shape, 'y_trainB.shape:', y_trainB.shape)

print('x_testA.shape:', x_testA.shape, 'y_testA.shape:', y_testA.shape)
print('x_testB.shape:', x_testB.shape, 'y_testB.shape:', y_testB.shape)



Les deux jeux de donnÃ©es contiennent 2000 images, de 64x64x3 pixels, rÃ©parties en quatre classes. 1500 images sont utilisÃ©es pour l'entrainement, 500 pour l'Ã©valuation. Seul le jeu de donnÃ©es A est complÃ¨tement annotÃ©, le jeu de donnÃ©e B ne contient que trÃ¨s peu d'annotations, correspondantent aux 50 premiers exemples de x_trainB.

<h2> Partie 1 : ModÃ¨le convolutionnel pour la classification d'images </h2>

On dÃ©finie ci-dessous une architecture convolutionnelle simple, Ã  ne pas modifier. Cette partie vise Ã  Ã©valuer la performance d'un modÃ¨le CNN sur les deux dataset A et B. Entrainez ce modÃ¨le des diffÃ©rentes maniÃ¨res possibles et reportez les performances obtenues dans le tableau situÃ© en fin de cette partie. A chaque nouvel apprentissage, utilisez les configurations suivantes : 
- 30 epochs
- Taille des minibatchs Ã  64
- 10% des donnÃ©es pour validation

Tracez les courbes d'apprentissage, et commentez vos rÃ©sultats.

In [0]:
# Couches d'apprentissage de reprÃ©sentations par convolutions 
ximg = Input(shape=(x_trainA.shape[1], x_trainA.shape[2], x_trainA.shape[3]))

l = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(ximg)
l = BatchNormalization(axis=-1)(l)
l = Activation('relu')(l)
l = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(l)
l = BatchNormalization(axis=-1)(l)
l = Activation('relu')(l)
l = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(l)
l = BatchNormalization(axis=-1)(l)
l = Activation('relu')(l)
l = AveragePooling2D((4,4))(l)
l = Conv2D(256, kernel_size=(3,3), strides=(2,2), padding='same')(l)
l = BatchNormalization(axis=-1)(l)
l = Conv2D(256, kernel_size=(3,3), strides=(1,1), padding='same')(l)
l = AveragePooling2D((2,2))(l)
l = Activation('relu')(l)
l = Dropout(0.2)(l)
feat_layer = Flatten()(l)

# conv_model(ximg) retourne la couche 'feat_layer' : 4096 dimensions pour une image 64x64x3
conv_model = Model(ximg, feat_layer) 

# Couches de classification
xfeat = Input(shape=(4096,))
y = Dense(1024, activation='relu')(xfeat)
y = Dropout(0.5)(y)
y = Dense(512, activation='relu')(y)
out_layer = Dense(num_classes, activation='softmax')(y)

# classif_model(xfeat) renvoie la couche sortie du classifieur : 4 dimensions
classif_model = Model(xfeat, out_layer) 

# ModÃ¨le CNN : reprÃ©sentation + classifier
feat = conv_model(ximg)
clf = classif_model(feat)
model = Model(ximg, clf)
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])
print(model.summary())

<h4> Entrainement du modÃ¨le sur les donnÃ©es A, puis Ã©valuation sur A et B</h4>

In [0]:
# A complÃ©ter 

Commentaires :

<h4> Entrainement et Ã©valuation du modÃ¨le sur les donnÃ©es B</h4>

<i>Attention Ã  ne pas rÃ©utiliser la variable <b>model</b>, dÃ©jÃ  entrainÃ©e sur A, il faut crÃ©er une nouvelle instance <b>model2</b> Ã  partir des fonctions <b>conv_model()</b> et <b>classif_model()</b> dÃ©finient prÃ©cÃ©demment.</i>

In [0]:
# A complÃ©ter 

Commentaires :

<h4>Finetuning sur B du modÃ¨le entrainÃ© sur A</h4>

In [0]:
# A complÃ©ter 

Commentaires :

<table>
  <thead>
    <th></th><th>Accuracy train A</th><th>Accuracy dev A</th><th>Accuracy test A</th><th>Accuracy train B</th><th>Accuracy dev B</th><th>Accuracy test B</th>
  </thead>
  <tbody>
    <tr><th>Entrainement sur A</th><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td></tr>
    <tr><th>Entrainement sur B</th><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td></tr>
    <tr><th>Entrainement sur A <br/>+ Finetuning sur B</th><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td><td>###</td></tr>
  </tbody>
</table>

Commentaires :




<h2>Partie 2 : Adaptation de domaine par alignement des activations</h2>

Cette partie consiste Ã  implÃ©menter et Ã©valuer un modÃ¨le inspirÃ© de l'article <a href="https://arxiv.org/pdf/1607.01719.pdf">Deep CORAL: Correlation Alignment for Deep Domain Adaptation [Sun & Saenko, 2016] </a>, dont la figure suivante illustre le fonctionnement. 

<center><img src="http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/coral16.png" width="50%" /></center>

ConsidÃ©rez les instructions suivantes :
- Utilisez les mÃªmes modules de convolutions que dans la Partie 1 afin de rester comparable. Pour cela, considÃ©rez une nouvelle instance <b>conv_model3</b> comme effectuÃ© prÃ©cÃ©demment.
- Seules les couches de convolutions sont partagÃ©es.
- Seulement deux couches denses aprÃ¨s les convolutions (ie: fc6 et fc7), dropout entre les deux.
- Version simplifiÃ©e de la <i>CORAL loss</i> : minimisation de la distance entre les derniÃ¨res couches denses (et/ou maximisation de la corrÃ©lation). Utilisez l'une des couches documentÃ©es sur la page suivante : https://keras.io/layers/merge/ 
- Entrainez pendant 50 epochs, avec minibatchs = 128


<h4> DÃ©finition du modÃ¨le </h4>

xA et xB sont les donnÃ©es des deux domaines, yA est la sortie du classifieur pour les donnÃ©es de xA dans le domaine source, et <i>coral</i> correspond Ã  la mÃ©trique (distance ou corrÃ©lation) entre un minibatch d'exemples xA et xB, dans les deux domaines. Le modÃ¨le optimise les deux loss : <i>categorical_crossentropy</i> pour classer les images du domaine source ; et <i>mse</i> qui minimise (ou maximise) la mÃ©trique considÃ©rÃ©e.


In [0]:
xA = Input(shape=(x_trainA.shape[1], x_trainA.shape[2], x_trainA.shape[3]))
xB = Input(shape=(x_trainA.shape[1], x_trainA.shape[2], x_trainA.shape[3]))

# A complÃ©ter

model4 = Model([xA,xB],[yA, coral])
model4.compile(loss=['categorical_crossentropy','mse'], optimizer=Adam(0.0001), metrics=['accuracy'])
print(model4.summary())

<h4> Entrainement du modÃ¨le d'adaptation par alignement </h4>

Selon le critÃ¨re <i>coral</i> que vous optimisez, considÃ©rez le vecteur objectif ne contenant que des 1 ou que des 0..

In [0]:
ones = np.ones((len(x_trainA),1)) # ou zeros = np.zeros((len(x_trainA),1))

# A complÃ©ter

Commentaires :

<h4>Evaluation sur les datasets A et B</h4>

In [0]:
# A complÃ©ter

Commentaires :

<h2>Partie 3 : Adaptation de domaine par contrainte adversarial</h2>

Cette partie consiste Ã  implÃ©menter un modÃ¨le d'adaptation de modÃ¨le par contrainte adversarial, inspirÃ© de l'article <a href="http://openaccess.thecvf.com/content_cvpr_2017/papers/Tzeng_Adversarial_Discriminative_Domain_CVPR_2017_paper.pdf">Adversarial Discriminative Domain Adaptation [Tzeng, 2017]</a>.

<center><img src="http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/adversadapt17.png" width="80%" /></center>

Dans la figure ci-dessus, les modules en pointillÃ©s indiquent une rÃ©utilisation de modÃ¨les dÃ©jÃ  entrainÃ©s, tandis que les modules en lignes pleines dÃ©signent des modÃ¨les nouvellement entrainÃ©s. Pour la phase de </i>Pre-training</i>, considÃ©rez le modÃ¨le entrainÃ© sur le dataset A dans la partie 1. 

La stratÃ©gie consiste donc Ã  :
- Faire le prÃ©training du modÃ¨le source sur les donnÃ©es source avec un critÃ¨re de classification. Cette Ã©tape a Ã©tÃ© faite en partie 1 : <b>conv_model</b> et <b>classif_model</b> sont dÃ©jÃ  entrainÃ©s.
- Ne pas rÃ©entrainer ces deux modules.
- Apprendre from scratch le modÃ¨le extracteur de caractÃ©ristiques pour les donnÃ©es cible Ã  l'aide d'un discriminateur adversarial appris Ã  discriminer entre les donnÃ©es source transformÃ©es par le modÃ¨le prÃ©appris et les donnÃ©es du domaine cible transformÃ©es par le modÃ¨le cible.  
- Construire un classifieur en empilant la couche de classification du modÃ¨le source sur l'extracteur de caractÃ©ristiques sur les donnÃ©es cible.



<h4> DÃ©finition du modÃ¨le discriminateur </h4>
Le discriminateur Ã  pour objectif de distinguer les domaines de deux sources de donnÃ©es (A et B) Ã  parir de reprÃ©sentations extraites de couches de convolutions. Ecrivez un modÃ¨le simple Ã  deux couches cachÃ©es. 

In [0]:
# A complÃ©ter

<h4> ModÃ¨le adversarial </h4>

Ecrivez ci-dessous le modÃ¨le adversarial qui combine le module <i>Target CNN</i> et le discriminateur. <i>Target CNN</i> consistera en une nouvelle instance <b>conv_model4</b> pour extraire des reprÃ©sentations similaires au modÃ¨le entrainÃ© en Partie 1 sur les images sources.

In [0]:
# A complÃ©ter

<h4> ModÃ¨le utilisÃ© en phase de test </h4>

Construisez ci-dessous le modÃ¨le final qui combine les modules <i>Target CNN</i>  (<b>conv_model4</b>) entrainÃ©s prÃ©cÃ©demment et <b>classif_model</b> entrainÃ© en Partie 1.

In [0]:
# A complÃ©ter

<h4> Entrainement du modÃ¨le adversarial</h4>

Utilisez les fonctions de crÃ©ation de minibatchs suivantes qui randomisent les ensembles d'apprentissage Ã  chaque nouvel epoch.


In [0]:
batch_size = 64
nbdata = len(x_trainA)

def get_batchA():
    global x_trainA, y_trainA
    i = 0
    while True:
        i = i + batch_size
        if i+batch_size > nbdata:
            i = 0
            lidx = list(range(nbdata))
            shuffle(lidx)
            x_trainA = x_trainA[lidx]
            y_trainA = y_trainA[lidx]
        yield x_trainA[i:i+batch_size], y_trainA[i:i+batch_size]

def get_batchB():
    global x_trainB, y_trainB
    i = 0
    while True:
        i = i + batch_size
        if i+batch_size > nbdata:
            i = 0
            lidx = list(range(nbdata))
            shuffle(lidx)
            x_trainB = x_trainB[lidx]
        yield x_trainB[i:i+batch_size]

data_genA = get_batchA()
data_genB = get_batchB()

<h4> Boucle d'apprentissage </h4>

In [0]:
ones = np.ones((batch_size,1))
zeros = np.zeros((batch_size,1))
nb_batchs = int(len(x_trainA)/batch_size)

for epoch in range(30):
    loss = 0.0
    acc = 0.0
    for batch in range(nb_batchs):
        # get minibatchs
        xA, yA = next(data_genA)
        xB = next(data_genB)

        # A complÃ©ter
        
        # train discriminator
        
        # train adversarial model on new minibatch
        
        # monitoring
        


Commentaires :

<h4> Evaluation sur les dataset A et B </h4>

In [0]:
# A complÃ©ter

Commentaires :

<hr/>

<b>Rappel, pour l'anonymisation de votre rendu, veuillez suivre les consignes suivantes : </b>

- Choisissez un numÃ©ro alÃ©atoire Ã  6 chiffres, le plus alÃ©atoire possible..
- Ecrivez ce numÃ©ro sur une feuille, ainsi que votre nom et numÃ©ro d'Ã©tudiant 
- Pliez cette feuille en 4, et donnez la Ã  la fin de l'examen au moment d'Ã©marger  
- Zippez votre fichier notebook, et nommez l'archive avec votre numÃ©ro alÃ©atoire 
- Envoyez l'archive via la page : http://stephane.ayache.perso.luminy.univ-amu.fr/examdeep/upload/upload.php
